In [1]:
import pyTigerGraph as tg

hostName = "https://director-lisdatasolutions.i.tgcloud.io"
graphName = "transport_demo"
secret = "kafpkc24og8jumuqcj8asnt2cm2g4h09"
userName = "user_1"
password = "A1z2e3r4*"
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password)
conn.getToken(secret)

c:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\.venv\Lib\site-packages\pyTigerGraph\pyTigerGraphAuth.py:264: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  datetime.utcfromtimestamp(float(res.get("expiration"))).strftime('%Y-%m-%d %H:%M:%S')


('m078mm1echr0nieho5rdb7rc8icuosln', 1732806365, '2024-11-28 15:06:05')

# Count distribute to 

In [6]:
result = conn.getEdgeCount("distribute_to")

In [7]:
result

29

In [12]:
results_test = conn.runInstalledQuery("get_distribute_to")

In [15]:
results_test[0]["@@total_nodes"]

26

In [19]:
results_test = conn.runInstalledQuery("capacity_total_distribute_to")

In [25]:
result_query = conn.runInstalledQuery("stock_total_nodes")


In [26]:
result_query

[{'@@total_stock': 1494.1}]

In [39]:
vertex_type = "Nodes"
result_query = conn.runInstalledQuery("get_quantity_of_nodes",params={"v_type_set" : vertex_type})


In [47]:
edge_type = "distribute_to"

In [49]:
result_query = conn.runInstalledQuery("capacity_average_edge",params={"e_type_set" : edge_type})

In [2]:

import concurrent.futures
from fastapi import HTTPException
from fastapi.responses import JSONResponse

def get_num_edges(conn,edge_type):
    """
    Get the count of edges for a given edge name.

    Parameters:
        conn: The connection object to the database.
        edge_name: The name of the edge.

    Returns:
        int: The number of edges.
    """
    result_query = conn.runInstalledQuery("get_quantity_of_edges",params={"e_type_set" : edge_type})
    return result_query[0]["@@edge_count"]

def get_num_nodes(conn,vertex_type):
    """
    Get the count of nodes for a given node name.

    Parameters:
        conn: The connection object to the database.
        node_name: The name of the node.

    Returns:
        int: The number of nodes.
    """
    result_query = conn.runInstalledQuery("get_quantity_of_nodes",params={"v_type_set" : vertex_type})
    return result_query[0]["@@total_nodes"]

def get_avg_capacity(conn,edge_type):
    """
    Get the average capacity of a given edge.

    Parameters:
        conn: The connection object to the database.
        edge_name: The name of the edge.

    Returns:
        float: The average capacity 
    """
    result_query = conn.runInstalledQuery("capacity_average_edge",params={"e_type_set" : edge_type})    
    return result_query[0]["average_capacity_edge"]

def get_total_capacity_edges(conn,edge_type):
    """
    Get the total capacity of all edges of a given edge type.

    Parameters:
        conn: The connection object to the database.
        edge_name: The name of the edge.

    Returns:
        float: The total capacity of the edges divided by 2.
    """
    result_query = conn.runInstalledQuery("capacity_total_edges",params={"e_type_set" : edge_type})
    return result_query[0]["@@total_capacity_edge"]

def get_total_capacity_nodes(conn,node_type):
    """
    Get the total capacity of all nodes of a given node type.

    Parameters:
        conn: The connection object to the database.
        node_name: The name of the node.

    Returns:
        float: The total capacity of the nodes, rounded to 1 decimal.
    """
    result_query = conn.runInstalledQuery("capacity_total_nodes",params={"v_type_set" : node_type})
    return result_query[0]["@@total_capacity"]

def get_total_stock(conn,vertex_type):
    """
    Get the total stock of all nodes of a given node type.

    Parameters:
        conn: The connection object to the database.
        node_name: The name of the node.

    Returns:
        float: The total stock of the nodes, rounded to the nearest integer.
    """
    result_query = conn.runInstalledQuery("stock_total_nodes",params={"v_type_set" : vertex_type})
    return result_query[0]["@@total_stock"]

def get_capacity_per_edge(total_capacity_warehouses, num_edges):
    """
    Calculate the capacity per edge.

    Parameters:
        total_capacity_warehouses: The total capacity of the warehouses.
        num_edges: The number of edges.

    Returns:
        float: The capacity per edge, rounded to the nearest integer.
    """
    if num_edges == 0:
        return 0
    return round(total_capacity_warehouses / num_edges, 0)

def get_KPI(conn, edge_type, vertex_type):
    """
    Get various Key Performance Indicators (KPIs) related to edges and nodes.

    Parameters:
        conn: The connection object to the database.
        edge_name: The name of the edge.
        node_name: The name of the node.

    Returns:
        dict: A dictionary containing various KPI results.
    """
    result_query = conn.runInstalledQuery("get_all_KPIs",params={"v_type_set" : vertex_type,"e_type_set":edge_type})
    return result_query

def get_kpi(edge_name: str, node_name: str):
    """
    Endpoint to retrieve KPI metrics for the specified edge and node.
    """
    try:
        kpi_results = get_KPI(conn, edge_name, node_name)
        return JSONResponse(content=kpi_results)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [4]:
get_KPI(conn,edge_type="distribute_to",vertex_type="Nodes")

[{'num_edges': 26},
 {'num_nodes': 29},
 {'avg_capacity': 61.55172},
 {'total_capacity_edges': 1785},
 {'total_capacity_warehouses': 1896.4},
 {'total_stock': 1494.1}]